In [24]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

def load_csv_dataset(csv_path):
    # Load data
    data = pd.read_csv(csv_path)

    # Separate filename, features, and labels
    filenames = data.iloc[:, 0]  # First column is filenames
    X = data.iloc[:, 1:-1]       # Middle columns are features
    y = data.iloc[:, -1]         # Last column is labels

    # Encode labels to 0-based integers
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)
    num_classes = len(label_encoder.classes_)

    # Split data
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # Convert to numpy arrays
    X_train = X_train.values.astype('float32')
    X_val = X_val.values.astype('float32')

    return X_train, X_val, y_train, y_val, num_classes

def create_model(input_shape, num_classes):
    model = models.Sequential([
        layers.Input(shape=input_shape),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

def train_and_evaluate(csv_path, epochs=50):
    # Load data
    X_train, X_val, y_train, y_val, num_classes = load_csv_dataset(csv_path)

    # Check for valid class distribution
    if num_classes < 2:
        raise ValueError("Dataset must contain at least two classes for classification")

    # Validate label ranges
    if np.max(y_val) >= num_classes:
        raise ValueError(f"Validation labels have values outside the class range [0, {num_classes-1}]")

    # Print dataset statistics
    print(f"Training samples: {len(X_train)}, Validation samples: {len(X_val)}")
    print(f"Number of classes: {num_classes}")

    # Define optimizers
    optimizers = {
        'Adam': tf.keras.optimizers.Adam(learning_rate=0.001),
        'SGD': tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9),
        'RMSprop': tf.keras.optimizers.RMSprop(learning_rate=0.001),
        'Adagrad': tf.keras.optimizers.Adagrad(learning_rate=0.01)
    }

    results = {}

    for name, optimizer in optimizers.items():
        print(f"\nTraining with {name} optimizer...")

        # Create and compile model
        model = create_model(input_shape=(X_train.shape[1],), num_classes=num_classes)
        model.compile(optimizer=optimizer,
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])

        # Train model
        history = model.fit(X_train, y_train,
                            epochs=epochs,
                            validation_data=(X_val, y_val),
                            batch_size=32,
                            verbose=1)

        # Evaluate model
        test_loss, test_accuracy = model.evaluate(X_val, y_val, verbose=0)
        y_pred = np.argmax(model.predict(X_val), axis=1)

        # Calculate metrics
        precision = precision_score(y_val, y_pred, average='macro')
        recall = recall_score(y_val, y_pred, average='macro')
        f1 = f1_score(y_val, y_pred, average='macro')

        # Store results
        results[name] = {
            'Test Loss': test_loss,
            'Test Accuracy': test_accuracy,
            'Precision': precision,
            'Recall': recall,
            'F1-Score': f1
        }

    # Print results
    print("\nFinal Results:")
    for opt, metrics in results.items():
        print(f"\n{opt} Optimizer:")
        for metric, value in metrics.items():
            print(f"{metric}: {value:.4f}")

# Run evaluation
train_and_evaluate("/content/Custom_CNN_Features.csv", epochs=10)

Training samples: 2602, Validation samples: 651
Number of classes: 1565

Training with Adam optimizer...
Epoch 1/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.4241 - loss: 36.1861 - val_accuracy: 0.4839 - val_loss: 3.8563
Epoch 2/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5313 - loss: 3.7839 - val_accuracy: 0.4839 - val_loss: 3.8862
Epoch 3/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5131 - loss: 3.8546 - val_accuracy: 0.4839 - val_loss: 3.8927
Epoch 4/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5250 - loss: 3.6891 - val_accuracy: 0.4839 - val_loss: 3.9309
Epoch 5/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.5250 - loss: 3.6653 - val_accuracy: 0.4839 - val_loss: 3.9649
Epoch 6/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.5195 - loss: 3.6988 - val_accuracy: 0.4839 - val_loss: 3.9843
Epoch 7/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5286 - loss: 3.5598 - val_accuracy: 0.4839 - val_loss: 4.0081
Epoch 8

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.2624 - loss: 18401132544.0000 - val_accuracy: 0.4823 - val_loss: 5.1730
Epoch 2/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5086 - loss: 39.5161 - val_accuracy: 0.4823 - val_loss: 5.3917
Epoch 3/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5317 - loss: 4.3895 - val_accuracy: 0.4823 - val_loss: 5.1496
Epoch 4/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.5148 - loss: 4.4317 - val_accuracy: 0.4823 - val_loss: 5.1000
Epoch 5/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.5288 - loss: 4.2946 - val_accuracy: 0.4823 - val_loss: 5.1114
Epoch 6/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.5418 - loss: 4.1711 - val_accuracy: 0.4823 - val_loss: 5.2462
Epoch 7/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5260 - loss: 4.2962 - val_accuracy: 0.4823 - val_loss: 5.0899
Epoch 8/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5201 - loss: 4.2868 - val_accuracy: 0.4823 - val_lo

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.4419 - loss: 37.7965 - val_accuracy: 0.4839 - val_loss: 3.8340
Epoch 2/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5384 - loss: 4.2032 - val_accuracy: 0.4839 - val_loss: 3.8383
Epoch 3/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5208 - loss: 3.7760 - val_accuracy: 0.4839 - val_loss: 3.8462
Epoch 4/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5099 - loss: 3.7781 - val_accuracy: 0.4839 - val_loss: 3.8833
Epoch 5/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5339 - loss: 3.5543 - val_accuracy: 0.4839 - val_loss: 3.8745
Epoch 6/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.5208 - loss: 3.5915 - val_accuracy: 0.4839 - val_loss: 3.8792
Epoch 7/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.5418 - loss: 3.4525 - val_accuracy: 0.4839 - val_loss: 3.8983
Epoch 8/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.5230 - loss: 3.5806 - val_accuracy: 0.4839 - val_loss: 3.910

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5006 - loss: 30.1864 - val_accuracy: 0.4839 - val_loss: 3.8170
Epoch 2/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5213 - loss: 3.8920 - val_accuracy: 0.4839 - val_loss: 3.8106
Epoch 3/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5363 - loss: 3.6891 - val_accuracy: 0.4839 - val_loss: 3.8095
Epoch 4/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.5312 - loss: 3.5157 - val_accuracy: 0.4839 - val_loss: 3.8121
Epoch 5/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.5230 - loss: 3.5639 - val_accuracy: 0.4839 - val_loss: 3.8158
Epoch 6/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5324 - loss: 3.5138 - val_accuracy: 0.4839 - val_loss: 3.8151
Epoch 7/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5414 - loss: 3.4194 - val_accuracy: 0.4839 - val_loss: 3.8125
Epoch 8/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5216 - loss: 3.5442 - val_accuracy: 0.4839 - val_loss: 3.8140

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
